<a href="https://colab.research.google.com/github/oziris/DL4CV/blob/master/DL4CV_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural Networks and Deep Learning is a free online book. The book will teach you about:

*   Neural networks, a beautiful biologically-inspired programming paradigm which enables a computer to learn from observational data
*   Deep learning, a powerful set of techniques for learning in neural networks

Neural networks and deep learning currently provide the best solutions to many problems in image recognition, speech recognition, and natural language processing. This book will teach you many of the core concepts behind neural networks and deep learning.
For more details about the approach taken in the book, see here. Or you can jump directly to Chapter 1 and get started.

Book: http://neuralnetworksanddeeplearning.com/ 

Code: https://github.com/chengfx/neural-networks-and-deep-learning-for-python3

In [0]:
# Standard library
import random

# Third-party libraries
import numpy as np

In [0]:
class Network(object):
	def __init__(self,sizes): #the list sizes contains the number of neurons in the respective layers.
		self.num_layers = len(sizes)  #the number of the layers in Network
		self.sizes = sizes
		self.biases = [np.random.randn(y,1) for y in sizes[1:]]
		self.weights = [np.random.randn(y,x) 
						for x,y in zip(sizes[:-1],sizes[1:])]

	def feedforward(self,a):
		"""Return the output of the network if "a" is input"""
		for b,w in zip(self.biases,self.weights):
			a = sigmoid(np.dot(w,a) + b)		
		return a
		
	def SGD(self, training_data, epochs, mini_batch_size,eta,
	        test_data = None):
		"""
		Train the neural network using mini-batch stochastic gradient descent.
		The "training_data" is a list of tuples "(x,y)" representing the training inputs
		and the desired output. The other non-optional parameters are self-explanatory.
		If "test_data" is provided then the network will be evaluated against the test 
		data after each epoch, and partial progress printed out. This is useful for tracking
		process, but slows things down substantially.
		"""
		if test_data is not None: 
			n_test = len(test_data)
		n = len(training_data)
		for j in range(epochs):
			random.shuffle(training_data)        #rearrange the training_data randomly
			mini_batches = [ training_data[k:k + mini_batch_size]
			                               for k in range(0, n, mini_batch_size)]
			for mini_batch in mini_batches:
				self.update_mini_batch(mini_batch,eta)
			if test_data:
				print("Epoch {0}: {1} / {2}".format(j,self.evaluate(test_data),n_test))
			else:
				print("Epoch {0} complete".format(j))
			
	def update_mini_batch(self,mini_batch,eta):
		"""
		Update the network's weights and biases by applying gradient descent using backpropagation
		to a single mini batch. The "mini_batch" is a list of tuples "(x,y)" and "eta" is the learning
		rate.
		"""
		nabla_b = [np.zeros(b.shape) for b in self.biases]
		nabla_w = [np.zeros(w.shape) for w in self.weights]
		for x, y in mini_batch:
			delta_nabla_b, delta_nabla_w = self.backprop(x, y)
			nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
			nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
		self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
		self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
	
	def backprop(self, x, y):
		"""Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
		nabla_b = [np.zeros(b.shape) for b in self.biases]
		nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
		activation = x
		activations = [x] # list to store all the activations, layer by layer
		zs = [] # list to store all the z vectors, layer by layer
		for b, w in zip(self.biases, self.weights):
			z = np.dot(w, activation)+b
			zs.append(z)
			activation = sigmoid(z)
			activations.append(activation)
        # backward pass
		delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
		nabla_b[-1] = delta
		nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
		for l in range(2, self.num_layers):
			z = zs[-l]
			sp = sigmoid_prime(z)
			delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
			nabla_b[-l] = delta
			nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
		return (nabla_b, nabla_w)

	def evaluate(self, test_data):
		"""Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
		test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
		return sum(int(x == y) for (x, y) in test_results)

	def cost_derivative(self, output_activations, y):
		"""Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
		return (output_activations-y)

In [0]:
#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [16]:
import tensorflow as tf
from keras.utils.np_utils import to_categorical   

mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images / 255.0
test_images=test_images/255.0

print(training_images.shape)
print(training_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [26]:
training_images_aux = [np.reshape(x, (784, 1)) for x in training_images]
test_images_aux = [np.reshape(x, (784, 1)) for x in test_images]
training_labels_aux = [np.reshape(to_categorical(y, num_classes=10), (10,1)) for y in training_labels]

print(training_images_aux[0].shape)
print(training_labels_aux[0].shape)
print(test_images_aux[0].shape)

(784, 1)
(10, 1)
(784, 1)


In [27]:
training_data = list(zip(training_images_aux, training_labels_aux))
test_data = list(zip(test_images_aux, test_labels))

net = Network([784, 30, 10])

net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9111 / 10000
Epoch 1: 9229 / 10000
Epoch 2: 9329 / 10000
Epoch 3: 9352 / 10000
Epoch 4: 9409 / 10000
Epoch 5: 9408 / 10000
Epoch 6: 9389 / 10000
Epoch 7: 9403 / 10000
Epoch 8: 9455 / 10000
Epoch 9: 9422 / 10000
Epoch 10: 9465 / 10000
Epoch 11: 9450 / 10000
Epoch 12: 9469 / 10000
Epoch 13: 9457 / 10000
Epoch 14: 9479 / 10000
Epoch 15: 9478 / 10000
Epoch 16: 9469 / 10000
Epoch 17: 9485 / 10000
Epoch 18: 9477 / 10000
Epoch 19: 9459 / 10000
Epoch 20: 9463 / 10000
Epoch 21: 9464 / 10000
Epoch 22: 9466 / 10000
Epoch 23: 9497 / 10000
Epoch 24: 9456 / 10000
Epoch 25: 9492 / 10000
Epoch 26: 9467 / 10000
Epoch 27: 9460 / 10000
Epoch 28: 9478 / 10000
Epoch 29: 9492 / 10000


In [28]:
idx = 88
print("Label:", test_labels[idx])
net.feedforward(test_images_aux[idx])

Label: 6


array([[9.10741863e-06],
       [6.85184195e-08],
       [5.32040531e-07],
       [5.71567368e-10],
       [1.83839676e-05],
       [5.05825429e-11],
       [9.99981648e-01],
       [6.50114365e-09],
       [1.01372711e-06],
       [1.15202397e-04]])